In [1]:
import os

def processString(t_pool, d_pool, it, x, flag, i, l):
    position_no=[]
    doc_no=[]
    dlen = len(d_pool)
    for j in range(dlen):
        dilen = len(d_pool[j])
        for k in range(dilen):
            if t_pool[i][l] in d_pool[j][k]:
                doc_no.append(j + it)
                position_no.append(k + it)
    setdn = set(doc_no)
    l1 = len(doc_no)
    l2 = len(list(setdn))
    string = (str(x) + ' ' + str(l2) + ' '+ str(l2))
    x += 1
    y = 0
    for j1 in range(l1):
        dj1 = doc_no[j1]
        pj1 = position_no[j1]
        diff2 = position_no[j1] - position_no[j1-1]
        diff1 = doc_no[j1] - doc_no[j1-1]
        if flag == False:
            string += ' ' + str(dj1 + 1) + ',' + str(pj1 + 1)
        elif flag == True:
            y += 1
            if y == 1:        
                string += ' ' + str(dj1) + ',' + str(pj1)
            elif diff1 == 0:
                string += ' ' + str(diff1) + ',' + str(diff2)
            else:
                string += ' ' + str(diff1) + ',' + str(pj1)
    return str(string + '\n'), x

In [2]:
def doc_processing(t_pool, d_pool, fl):
    x1 = 0
    x = 0
    flag = False
    file = open(r"C:\Users\pc\New Folder\assignment1\doc_index","w")
    if fl == True:
        file = open(r"C:\Users\pc\New Folder\assignment1\Hash_doc_index","w")
        x1 = 1
        flag = True
    len11 = len(t_pool)
    for i1 in range(len11):
        len12 = len(t_pool[i1])
        for jj in range(len12):
            s1, x = processString(t_pool, d_pool, x1, x, flag, i1, jj)
            file.write(s1)
    file.close()            

In [3]:
def printDetails(index, word, path):
    print("\nThe word to find: ", word)
    print("\nTerm Index : ", str(index), "\n")
    file = open(path +'\doc_index', "r")
    for x in ( raw.strip().split() for raw in file ):
        if x[0] == index:
            print("Number of documents containing the word: ", str(x[2]), "\n")
            print("Number of times word repeated in the dataset: ", str(x[1]), "\n")
            return True
    return False

In [4]:
def SearchNLP(pool, text):
    path = r"C:\Users\pc\New Folder\assignment1"
    my_dic={}
    i = 0
    print("Assigning Term IDs in my_dic....................")
    for x in pool:
        for y in x: 
            my_dic.update({str(y):str(i)})
            i = i + 1
    doc_processing(pool, text, True)
    doc_processing(pool, text, False)
    
    flag = True
    while flag == True:
        print("\n\n\nEnter the word to search or enter -1 to halt: ")
        in_word = input()
        if in_word == "-1":
            print("\nSearch Halted!\n\nHave a nice day!")
            return
        dic_in = my_dic.get(in_word)
        if printDetails(dic_in, in_word, path) == False:
            print('Sorry, The word Entered word is not present\nPlease try a different word')

In [5]:
import os
import re
import math
from stemming.porter2 import stem
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#preprocessing data
#----------------------------------------------------------------------
def getFilesInDir(dir): 
	lst = []   
	for file in os.listdir(dir):
		lst.append(file)
	return lst

def getParsedDocs(dir):
	print(dir)
	files = getFilesInDir(dir)
	print("The files are as follow:\n")
	print(files)
	print("\n")    
	docs = []

	for file in files:      
		text = open(dir+"//"+file, 'r', encoding="utf8").read()
		docs.append(text)
	return docs

#tokenization--------------------------------------------------------------
def tokenize(docs):

	docs_tokens=[]
	for doc in docs:
		s = re.findall(r'\w+',doc)
		docs_tokens.append(s)

	temp_tokens = []
	pool = []
	for i in range(len(docs_tokens)):
		for j in range(len(docs_tokens[i])):
			docs_tokens[i][j] = docs_tokens[i][j].lower()
	return docs_tokens

#-------Remove stopwords
def removeStopWords(pool):

	file = open ("C:/Users/pc/New Folder/assignment1/stopwords.txt","r")
	content = file.readlines()

	temp_content = []

	for word in content:
		word = word[:-1]
		temp_content.append(word)
	stop_list = temp_content
    

	temp_pool = []
	for doc_tokens in pool:
		 for word in stop_list:
		 	if word in doc_tokens:
		 		removeAll(doc_tokens, word)
		 temp_pool.append(doc_tokens)
        
	pool = temp_pool 
	return pool

def removeEmptySpace(pool):
	return pool
	temp_pool = []
	print("Please Wait, this part takes the longest time")
	for doc_tokens in pool:
		c = ''
		while c in doc_tokens:
			index = doc_tokens.index(c)
			doc_tokens.pop(index)
		temp_pool.append(doc_tokens)
	return temp_pool

def removeAll(lst, token):
	while token in lst:
		lst.remove(token)
	return lst

#--------------Stemming function to reduce inflected words
def stemming(pool):
	temp_pool = []
	for doc_tokens in pool:
		temp_doc = []
		for token in doc_tokens:
			token = stem(token)
			temp_doc.append(token)
		temp_pool.append(temp_doc)
	return temp_pool	

#function to remove duplicate words
def removeDuplicates(pool):
	temp_pool = []
	for doc in pool:
		temp_pool.append(list(set(doc)))
	return temp_pool

#--------------------------------------------------------------
#Assigning termIDs to dictionary(Tokenized words)
def assignTermid(pool):
	file = open(r"TermID.txt","w+")
	i = 1
	for docs in pool:
		for token in docs: 
			file.write(str(i) + '\t' + token + "\n")
			i += 1
	file.close()
#--------------------------------------------------------------
#Assigning docID
def assignDocids(dir):
	file_dict = assignId(dir)
	file_names = getFilesInDir(dir)
	file = open(r"DocID.txt","w+")

	i = 0
	for name in file_names:
		file.write(name +"\t"+ str(file_dict[i][name]) + "\n")
		i += 1
	file.close()

def assignId(dir):
	file_names = getFilesInDir(dir)
	file_dict = []
	i = 0
	for file in file_names:
		i += 1
		file_dict.append({file : i})
	return file_dict

#--------------------------------------------------------------
#      INVERTED INDEX AND DOCUMENT FREQUENCY RECORDING
#--------------------------------------------------------------

def usingdict(pool):
	dic={}
	i = 1
	print("Assigning Term IDs in a dictionary")
	for docs in pool:
		for token in docs: 
			dic.update({str(token):str(i)})
			i += 1
	return dic


def computeWeight(termF,totalD,freqInDoc):
	if termF>0:    
		idff= math.log10(totalD/freqInDoc)
		weight= 1 + (math.log10(termF)) * idff
		idff=round(idff,3)
		weight=round(weight,3)
		return weight       
	else:
		return 0        

#	return weight

def normalizedWeight(array,maxWeight): 
    normalisedWeight=  (array/maxWeight)
    return np.round(normalisedWeight,3)


def cosineSimilarity(array):
	for i in array:
		for j in array:
			print(round(cosine_similarity(i,j)[0][0] ,3),end='\t')
		print('')        

def processPositions(token_pool, doc_pool):
	file = open(r"InvertedIndex.txt","w+")
	outputFormat="<TermID> <Token> <Frequency> <OccuranceInUniqueDocs> <DocNum : CountInThatDoc>\n"    
	file.write(outputFormat)
	m=0
	docLength=(len(token_pool))
	weightedWordsArray=[]  
	normalisedWordsArray=[]
	for i in range(len(token_pool)):
		for l in range(len(token_pool[i])):
			posno=[]
			docno=[]
			countss=[]
			inDocument=[]
			for j in range(len(doc_pool)):
				countInDoc=0
				for k in range(len(doc_pool[j])):
					if token_pool[i][l] in doc_pool[j][k]:
						docno.append(j)
						posno.append(k)
						countInDoc+=1
				countss.append(countInDoc)
				inDocument.append(j)
			setofno=list(set(docno))
			m=m+1   
			string = str((m)) + '  ' + token_pool[i][l]+ "  " + str(len(docno)) + '  '+ str(len(setofno)) + "\t"
			lendocno=len(docno)
			docFreqForWordI=setofno
			tempWeight=[]
			tempNormalised=[]
			for j in range(len(countss)):
				termFrequencyforij =countss[j]
				weightedWord=computeWeight(termFrequencyforij, docLength ,len(docFreqForWordI))
				tempWeight.append(weightedWord)
				string += '  ' + str(inDocument[j]+1) + ':' + str(countss[j]) + ","
			weightedWordsArray.append(tempWeight)
			string+='\n'
			file.write(string)
	LISTSpecial =  np.array(weightedWordsArray).T  
	normalisedWordsArray=[]
	for m in LISTSpecial:
		print("Max value in Doc : ",max(m))
		vec1 = np.array(normalizedWeight(m,max(m)))
		vec1 = np.reshape(vec1, (-1, len(vec1)))
		normalisedWordsArray.append(vec1)
	print("\n\nNormalised Document Vectors Are:\n")       
	print(normalisedWordsArray)
	print("\n\nYour Doc-Doc incidence matrix is:\n")  
	cosineSimilarity(normalisedWordsArray)           
	file.close()            
            
#==================== MAIN PROGRAM
def main():
	#dir2=argv[-1] 
	dir = r"C:/Users/pc//New Folder/assignment1/dataset/"
    
	docs = getParsedDocs(dir)
	print("Obtaining files from Data Set...\n")
	print(docs[0][-100:])
    
	pool = tokenize(docs)
	text = pool
	print("Tokenization...\n")    
	print("\nEliminating Stopwords\n")
	pool = removeStopWords(pool)    
	print("\nStemming...\n")    
	pool = stemming(pool) 
	print("\nRemoving Duplicates...\n")
	pool = removeDuplicates(pool)
	print("\nAssigning Term IDs...\n")
	assignTermid(pool)   
	print("\nAssigning Document IDs...\n")  
	assignDocids(dir)

#-------------------------------------------------------

	dic=usingdict(pool)
	processPositions(pool, text)
	computeWeight(3,10,3)
	SearchNLP(pool, text)
	return


main()	



C:/Users/pc//New Folder/assignment1/dataset/
The files are as follow:

['a.txt', 'b.txt', 'c.txt']


Obtaining files from Data Set...

pers, ‘--Mystery, ancient and modern, with
Seaography: then Drawling--the Drawling-master was an o


Tokenization...


Eliminating Stopwords


Stemming...


Removing Duplicates...


Assigning Term IDs...


Assigning Document IDs...

Assigning Term IDs in a dictionary
Max value in Doc :  1.436
Max value in Doc :  1.436
Max value in Doc :  1.61


Normalised Document Vectors Are:

[array([[0.696, 0.813, 0.696, ..., 0.   , 0.   , 0.696]]), array([[0.696, 0.813, 0.696, ..., 0.   , 0.   , 0.696]]), array([[0.621, 0.   , 0.621, ..., 0.   , 0.621, 0.621]])]


Your Doc-Doc incidence matrix is:

1.0	0.99	0.504	
0.99	1.0	0.506	
0.504	0.506	1.0	
Assigning Term IDs in my_dic....................



Enter the word to search or enter -1 to halt: 
lark

The word to find:  lark

Term Index :  None 

Sorry, The word Entered word is not present
Please try a different word
